In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# ──────────────────────────────────────────────
#  LOAD DATA
# ──────────────────────────────────────────────
df = pd.read_csv('data/matches.csv')
df['date'] = pd.to_datetime(df['date'])
df['toss_won_match'] = df['toss_winner'] == df['winner']

print(f"✅ Loaded {len(df)} matches across {df['season'].nunique()} seasons\n")
# ──────────────────────────────────────────────
#  GLOBAL STYLE
# ──────────────────────────────────────────────
sns.set_theme(style='darkgrid')
plt.rcParams.update({
    'figure.facecolor': '#0f0f0f',
    'axes.facecolor':   '#1a1a2e',
    'axes.labelcolor':  '#e0e0e0',
    'xtick.color':      '#aaaaaa',
    'ytick.color':      '#aaaaaa',
    'text.color':       '#e0e0e0',
    'grid.color':       '#2a2a3e',
    'axes.edgecolor':   '#2a2a3e',
    'font.size':        10,
})

GOLD   = '#FFD700'
COLORS = ['#e63946','#457b9d','#2a9d8f','#e9c46a',
          '#f4a261','#a8dadc','#8338ec','#06d6a0',
          '#fb8500','#219ebc']


In [ ]:
# ══════════════════════════════════════════════
#  Q1 — WHICH TEAM WINS MOST?
# ══════════════════════════════════════════════
wins = df['winner'].value_counts().head(10).reset_index()
wins.columns = ['team', 'wins']

fig, ax = plt.subplots(figsize=(12, 6), facecolor='#0f0f0f')

bars = ax.barh(wins['team'][::-1], wins['wins'][::-1],
               color=COLORS[:len(wins)], edgecolor='none', height=0.65)

for bar, val in zip(bars, wins['wins'][::-1]):
    ax.text(bar.get_width() + 0.2, bar.get_y() + bar.get_height() / 2,
            f'{val} wins', va='center', fontsize=10, fontweight='bold', color=GOLD)

ax.set_title('🏆  Which Team Wins Most?', fontsize=15, fontweight='bold',
             color=GOLD, pad=15)
ax.set_xlabel('Number of Wins', fontsize=11)
ax.set_xlim(0, wins['wins'].max() * 1.18)
ax.spines[['top', 'right', 'left']].set_visible(False)
plt.tight_layout()
plt.savefig('q1_team_wins.png', dpi=150, bbox_inches='tight', facecolor='#0f0f0f')
plt.show()
print("✅  Team wins chart saved")


In [ ]:
# ══════════════════════════════════════════════
#   DOES TOSS AFFECT MATCH RESULT?
# ══════════════════════════════════════════════
toss_overall = df['toss_won_match'].mean() * 100
toss_by_dec  = df.groupby('toss_decision')['toss_won_match'].mean() * 100

fig, axes = plt.subplots(1, 2, figsize=(13, 5), facecolor='#0f0f0f')
fig.suptitle('🎯  Does Toss Affect Match Result?', fontsize=15,
             fontweight='bold', color=GOLD, y=1.02)

# -- Pie chart
ax1 = axes[0]
pie_vals   = [toss_overall, 100 - toss_overall]
pie_labels = [f'Toss Winner\nWon ({toss_overall:.1f}%)',
              f'Toss Winner\nLost ({100-toss_overall:.1f}%)']
ax1.pie(pie_vals, labels=pie_labels, colors=[GOLD, '#333355'],
        startangle=90, autopct='%1.1f%%',
        textprops={'color': '#e0e0e0', 'fontsize': 11},
        wedgeprops={'edgecolor': '#0f0f0f', 'linewidth': 2.5})
ax1.set_title('Overall Toss → Win Rate', fontsize=12, color=GOLD, pad=10)

# -- Bar chart by decision
ax2 = axes[1]
dec_colors = ['#e63946' if d == 'bat' else '#457b9d' for d in toss_by_dec.index]
bars2 = ax2.bar(toss_by_dec.index, toss_by_dec.values,
                color=dec_colors, edgecolor='none', width=0.4)
for bar, val in zip(bars2, toss_by_dec.values):
    ax2.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.8,
             f'{val:.1f}%', ha='center', fontsize=12, fontweight='bold', color=GOLD)
ax2.axhline(50, color='#aaaaaa', linestyle='--', linewidth=1.3, label='50% baseline')
ax2.set_ylim(0, 100)
ax2.set_ylabel('Win % after Winning Toss', fontsize=11)
ax2.set_title('Win Rate: Bat vs Field First', fontsize=12, color=GOLD, pad=10)
ax2.legend(fontsize=9)
ax2.spines[['top', 'right', 'left']].set_visible(False)

# Insight text
insight = (" Toss has minimal impact — nearly 50/50.\n"
           "Batting or fielding first makes little difference.")
fig.text(0.5, -0.04, insight, ha='center', fontsize=10,
         color='#aaaaaa', style='italic')

plt.tight_layout()
plt.savefig('q2_toss_effect.png', dpi=150, bbox_inches='tight', facecolor='#0f0f0f')
plt.show()
print("/ Toss effect chart saved")

In [ ]:
# ══════════════════════════════════════════════
#  Q3 — WHICH STADIUM FAVORS WHICH TEAM?
# ══════════════════════════════════════════════
top_venues = df['venue'].value_counts().head(8).index
venue_df   = df[df['venue'].isin(top_venues)]

# Dominant team per venue
venue_wins = (venue_df.groupby(['venue', 'winner'])
              .size().reset_index(name='wins'))
dominant   = (venue_wins.sort_values('wins', ascending=False)
              .groupby('venue').first().reset_index()
              .sort_values('wins', ascending=True))

fig, ax = plt.subplots(figsize=(13, 6), facecolor='#0f0f0f')

bar_colors = [COLORS[i % len(COLORS)] for i in range(len(dominant))]
bars = ax.barh(dominant['venue'], dominant['wins'],
               color=bar_colors, edgecolor='none', height=0.6)

for bar, (_, row) in zip(bars, dominant.iterrows()):
    ax.text(bar.get_width() + 0.1, bar.get_y() + bar.get_height() / 2,
            f"  ← {row['winner']}  ({int(row['wins'])} wins)",
            va='center', fontsize=9.5, color='#e0e0e0')

ax.set_title('🏟️  Which Stadium Favors Which Team?', fontsize=15,
             fontweight='bold', color=GOLD, pad=15)
ax.set_xlabel('Wins by Dominant Team at Venue', fontsize=11)
ax.set_xlim(0, dominant['wins'].max() * 2.0)
ax.spines[['top', 'right', 'left']].set_visible(False)
plt.tight_layout()
plt.savefig('q3_stadium_favorites.png', dpi=150, bbox_inches='tight', facecolor='#0f0f0f')
plt.show()
print(" Stadium favorites chart saved")

# Heatmap bonus — top 6 teams vs top 8 venues
pivot = (venue_df[venue_df['winner'].isin(wins['team'].head(6))]
         .groupby(['winner', 'venue'])
         .size().unstack(fill_value=0))

fig, ax = plt.subplots(figsize=(13, 5), facecolor='#0f0f0f')
sns.heatmap(pivot, annot=True, fmt='d', cmap='YlOrRd',
            linewidths=0.5, linecolor='#0f0f0f',
            cbar_kws={'label': 'Wins'}, ax=ax)
ax.set_title('🔥  Team Wins Per Venue — Heatmap', fontsize=14,
             fontweight='bold', color=GOLD, pad=15)
ax.set_xlabel('Venue', fontsize=11)
ax.set_ylabel('Team', fontsize=11)
plt.xticks(rotation=30, ha='right', fontsize=8)
plt.tight_layout()
plt.savefig('q3_heatmap.png', dpi=150, bbox_inches='tight', facecolor='#0f0f0f')
plt.show()
print("Venue heatmap saved")

In [ ]:
# ══════════════════════════════════════════════
#  Q4 — WHO ARE TOP PERFORMERS?
# ══════════════════════════════════════════════
pom = df['player_of_match'].value_counts().head(12).reset_index()
pom.columns = ['player', 'awards']

# color: gold/silver/bronze for top 3, rest steel blue
bar_colors_pom = ([GOLD, '#C0C0C0', '#CD7F32'] +
                  ['#457b9d'] * (len(pom) - 3))

fig, ax = plt.subplots(figsize=(13, 6), facecolor='#0f0f0f')
bars = ax.bar(range(len(pom)), pom['awards'],
              color=bar_colors_pom, edgecolor='none', width=0.65)

for bar, val in zip(bars, pom['awards']):
    ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.05,
            str(val), ha='center', fontsize=10, fontweight='bold', color='#e0e0e0')

ax.set_xticks(range(len(pom)))
ax.set_xticklabels(pom['player'], rotation=35, ha='right', fontsize=9.5)
ax.set_ylabel('Player of the Match Awards', fontsize=11)
ax.set_title('⭐  Top Performers — Player of the Match Awards', fontsize=15,
             fontweight='bold', color=GOLD, pad=15)

# medal labels
for i, label in enumerate(['🥇 Gold', '🥈 Silver', '🥉 Bronze']):
    ax.text(i, pom['awards'].iloc[i] / 2, label,
            ha='center', fontsize=8, color='#0f0f0f', fontweight='bold')

ax.spines[['top', 'right', 'left']].set_visible(False)
plt.tight_layout()
plt.savefig('q4_top_performers.png', dpi=150, bbox_inches='tight', facecolor='#0f0f0f')
plt.show()
print(" Top performers chart saved")

In [ ]:
# ══════════════════════════════════════════════
#  SUMMARY PRINT
# ══════════════════════════════════════════════
print("\n" + "="*55)
print("         📊  IPL ANALYSIS — KEY FINDINGS")
print("="*55)

print("\n🏆  Most Wins:")
for _, r in wins.head(5).iterrows():
    bar = '█' * r['wins']
    print(f"   {r['team']:<32} {r['wins']:>2} wins")

print(f"\n🎯  Toss Effect:")
print(f"   Toss winner won match : {toss_overall:.1f}%")
for dec, pct in toss_by_dec.items():
    print(f"   Chose to {dec:<6} → won : {pct:.1f}%")
print(f"   Verdict: Toss has MINIMAL impact on result.")

print(f"\n🏟️  Stadium Favorites:")
for _, r in dominant.sort_values('wins', ascending=False).head(5).iterrows():
    print(f"   {r['venue'][:38]:<39} → {r['winner']}")

print(f"\n⭐  Top Performers (POTM Awards):")
for _, r in pom.head(5).iterrows():
    print(f"   {r['player']:<28} {r['awards']} awards")

print("\n" + "="*55)
print("All charts saved as PNG files in your working directory.")
print("="*55)